In [5]:
import pandas as pd
import os

In [10]:
# имя файла с данными
data_file_dir = u"./data"
data_file_names = [os.path.join(data_file_dir, x) for x in os.listdir(data_file_dir)]
data_file_names

['./data/test.csv', './data/train.csv', './data/gender_submission.csv']

In [12]:
# данные и их структура
df = pd.read_csv(data_file_names[1])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
